In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using Theano backend.


### Load training data

Load `train.csv` from Kaggle into a pandas DataFrame.

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Set up X and y

NOTE: Keras requires a `numpy` matrix, it doesn't work with `pandas`.

In [4]:
y= df['label']
X= df.drop(['label'], axis=1).values

In [5]:
type(X)

numpy.ndarray

### Preprocessing

1. When dealing with image data, you need to normalize your `X` by dividing each value by the max number of pixels (255).
2. Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix

In [6]:
y = to_categorical(y)

In [7]:
X = X/255.

In [8]:
X

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

### Train/Test Split

We want to create a validation set that the model will never see to approximate how it's going to do with Kaggle's `test.csv`. Use `sklearn`'s `train_test_split` to do this.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(31500, 784) (10500, 784) (31500, 10) (10500, 10)


### Create your neural network

Create a neural network using the `Dense` and `Dropout` layers from `keras`. Your activation function for the final output layer needs to be `softmax` to accomidate the ten different classes.

In [11]:
model = Sequential()
model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(.5))
model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))



### Compile your model

Since this is a multiclass classification problem, your loss function is `categorical_crossentropy`.

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit the model

Use your X_test, y_test from the `train_test_split` step for the `validation_data` parameter.

In [13]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Train on 31500 samples, validate on 10500 samples
Epoch 1/10
31500/31500 [==============================] - 7s 211us/step - loss: 0.3216 - acc: 0.9001 - val_loss: 0.1514 - val_acc: 0.9538
Epoch 2/10
31500/31500 [==============================] - 10s 313us/step - loss: 0.1621 - acc: 0.9488 - val_loss: 0.1291 - val_acc: 0.9592
Epoch 3/10
31500/31500 [==============================] - 11s 353us/step - loss: 0.1266 - acc: 0.9603 - val_loss: 0.1122 - val_acc: 0.9657
Epoch 4/10
31500/31500 [==============================] - 12s 366us/step - loss: 0.1041 - acc: 0.9675 - val_loss: 0.0951 - val_acc: 0.9707
Epoch 5/10
31500/31500 [==============================] - 12s 371us/step - loss: 0.0945 - acc: 0.9700 - val_loss: 0.0873 - val_acc: 0.9730
Epoch 6/10
31500/31500 [==============================] - 12s 383us/step - loss: 0.0822 - acc: 0.9728 - val_loss: 0.0966 - val_acc: 0.9731
Epoch 7/10
31500/31500 [==============================] - 12s 392us/step - loss: 0.0749 - acc: 0.9757 - val_loss: 0.0

### Load in Kaggle's `test.csv`

Be sure to do the **same** preprocessing you did for your training `X`.

In [16]:
test = pd.read_csv('test.csv')
test = test.values/255.

### Create your predictions

Use `predict_classes` to get the actual numerical values (0-9).

In [17]:
pred = model.predict_classes(test)

28000/28000 [==============================] - 1s 43us/step


In [18]:
pred

array([2, 0, 9, ..., 3, 9, 2])

### Prepare your submission

1. Add your predictions to a column called `Label`
2. You'll need to manually create the `ImageId` column, which is just a list of 1..[NUMBER OF TEST SAMPLES]

In [20]:
submission = pd.DataFrame()


submission['ImageId'] = range(1,test.shape[0] + 1)

submission['Label']= pred

In [23]:
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


### Create your submission csv

Remember to set `index=False`!

In [24]:
submission.to_csv('submission.csv', index=False)

In [25]:
# First score is 97.5%

# Second model

In [12]:
model = Sequential()
model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20)

Train on 31500 samples, validate on 10500 samples
Epoch 1/20
31500/31500 [==============================] - 10s 303us/step - loss: 0.2876 - acc: 0.9111 - val_loss: 0.1603 - val_acc: 0.9495
Epoch 2/20
31500/31500 [==============================] - 15s 469us/step - loss: 0.1234 - acc: 0.9629 - val_loss: 0.1188 - val_acc: 0.9644
Epoch 3/20
31500/31500 [==============================] - 17s 534us/step - loss: 0.0912 - acc: 0.9718 - val_loss: 0.1037 - val_acc: 0.9725
Epoch 4/20
31500/31500 [==============================] - 18s 585us/step - loss: 0.0655 - acc: 0.9810 - val_loss: 0.1038 - val_acc: 0.9728
Epoch 5/20
31500/31500 [==============================] - 19s 611us/step - loss: 0.0592 - acc: 0.9815 - val_loss: 0.1344 - val_acc: 0.9658
Epoch 6/20
31500/31500 [==============================] - 20s 632us/step - loss: 0.0458 - acc: 0.9857 - val_loss: 0.1107 - val_acc: 0.9719
Epoch 7/20
31500/31500 [==============================] - 21s 665us/step - loss: 0.0434 - acc: 0.9867 - val_loss: 0.

In [13]:
test = pd.read_csv('test.csv')
test = test.values/255.

pred = model.predict_classes(test)


28000/28000 [==============================] - 1s 49us/step


In [14]:
submission = pd.DataFrame()


submission['ImageId'] = range(1,test.shape[0] + 1)

submission['Label']= pred

In [15]:
submission.to_csv('submission2.csv', index=False)

In [16]:
# Second submission is 97.3%

# Third submission

In [19]:
model = Sequential()
model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=100)

Train on 31500 samples, validate on 10500 samples
Epoch 1/50
31500/31500 [==============================] - 4s 114us/step - loss: 0.2694 - acc: 0.9168 - val_loss: 0.1338 - val_acc: 0.9576
Epoch 2/50
31500/31500 [==============================] - 4s 115us/step - loss: 0.0996 - acc: 0.9706 - val_loss: 0.1148 - val_acc: 0.9656
Epoch 3/50
31500/31500 [==============================] - 4s 137us/step - loss: 0.0609 - acc: 0.9796 - val_loss: 0.1155 - val_acc: 0.9669
Epoch 4/50
31500/31500 [==============================] - 5s 143us/step - loss: 0.0420 - acc: 0.9862 - val_loss: 0.1039 - val_acc: 0.9708
Epoch 5/50
31500/31500 [==============================] - 5s 148us/step - loss: 0.0345 - acc: 0.9883 - val_loss: 0.1228 - val_acc: 0.9678
Epoch 6/50
31500/31500 [==============================] - 5s 156us/step - loss: 0.0276 - acc: 0.9907 - val_loss: 0.1106 - val_acc: 0.9736
Epoch 7/50
31500/31500 [==============================] - 5s 156us/step - loss: 0.0245 - acc: 0.9922 - val_loss: 0.1218 - 

In [20]:
test = pd.read_csv('test.csv')
test = test.values/255.

pred = model.predict_classes(test)

28000/28000 [==============================] - 1s 42us/step


In [21]:
submission = pd.DataFrame()


submission['ImageId'] = range(1,test.shape[0] + 1)

submission['Label']= pred

In [22]:
submission.to_csv('submission3.csv', index=False)

In [23]:
# It is 98 %

In [27]:
model = Sequential()
model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=100)


Train on 31500 samples, validate on 10500 samples
Epoch 1/40
31500/31500 [==============================] - 11s 350us/step - loss: 0.0052 - acc: 0.9987 - val_loss: 0.1919 - val_acc: 0.9779
Epoch 2/40
31500/31500 [==============================] - 11s 336us/step - loss: 0.0064 - acc: 0.9982 - val_loss: 0.1799 - val_acc: 0.9774
Epoch 3/40
31500/31500 [==============================] - 11s 342us/step - loss: 0.0071 - acc: 0.9986 - val_loss: 0.1956 - val_acc: 0.9773
Epoch 4/40
31500/31500 [==============================] - 11s 346us/step - loss: 0.0040 - acc: 0.9989 - val_loss: 0.1828 - val_acc: 0.9790
Epoch 5/40
31500/31500 [==============================] - 11s 351us/step - loss: 0.0023 - acc: 0.9994 - val_loss: 0.1860 - val_acc: 0.9777
Epoch 6/40
31500/31500 [==============================] - 11s 353us/step - loss: 0.0029 - acc: 0.9992 - val_loss: 0.1958 - val_acc: 0.9774
Epoch 7/40
31500/31500 [==============================] - 11s 341us/step - loss: 0.0036 - acc: 0.9992 - val_loss: 0.

In [30]:
test = pd.read_csv('test.csv')
test = test.values/255.

pred = model.predict_classes(test)

submission = pd.DataFrame()


submission['ImageId'] = range(1,test.shape[0] + 1)

submission['Label']= pred

28000/28000 [==============================] - 2s 60us/step


In [31]:
submission.to_csv('submission4.csv', index=False)

In [32]:
## Score is  0.97628

In [33]:
model = Sequential()
model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=100)


Train on 31500 samples, validate on 10500 samples
Epoch 1/30
31500/31500 [==============================] - 4s 137us/step - loss: 0.3558 - acc: 0.8911 - val_loss: 0.1535 - val_acc: 0.9511
Epoch 2/30
31500/31500 [==============================] - 4s 134us/step - loss: 0.1625 - acc: 0.9494 - val_loss: 0.1326 - val_acc: 0.9565
Epoch 3/30
31500/31500 [==============================] - 5s 153us/step - loss: 0.1242 - acc: 0.9610 - val_loss: 0.1149 - val_acc: 0.9643
Epoch 4/30
31500/31500 [==============================] - 5s 159us/step - loss: 0.1016 - acc: 0.9685 - val_loss: 0.0995 - val_acc: 0.9710
Epoch 5/30
31500/31500 [==============================] - 5s 164us/step - loss: 0.0911 - acc: 0.9718 - val_loss: 0.0913 - val_acc: 0.9718
Epoch 6/30
31500/31500 [==============================] - 5s 167us/step - loss: 0.0804 - acc: 0.9743 - val_loss: 0.0897 - val_acc: 0.9747
Epoch 7/30
31500/31500 [==============================] - 5s 172us/step - loss: 0.0732 - acc: 0.9762 - val_loss: 0.0914 - 

In [35]:
test = pd.read_csv('test.csv')
test = test.values/255.

pred = model.predict_classes(test)

submission = pd.DataFrame()


submission['ImageId'] = range(1,test.shape[0] + 1)

submission['Label']= pred

28000/28000 [==============================] - 2s 56us/step


In [36]:
submission.to_csv('submission5.csv', index=False)

In [ ]:
# Last score is 0.97885